In [144]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
import os
from scipy.spatial import distance_matrix
import pandas as pd
import seaborn as sns
import pickle

In [133]:
all_results = pd.read_csv('/Users/orestisbastas/Developer/bio2d_public/results/initial_results.csv')
all_results = all_results[all_results['model_type'] == 'catboost']

In [125]:
# Standardize features
scaler = StandardScaler()
def standardize(X):
    return scaler.fit_transform(X)

# PCA & t-SNE Visualization with Subplots
def plot_pca_tsne(X, ax1, ax2, ax3, ax4, spatial_distance, spatial_distance_max):
    X_scaled = standardize(X)
    
    # PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)
    ax1.scatter(X_pca[:, 0], X_pca[:, 1], c=y, cmap='coolwarm', alpha=0.7)
    ax1.set_xlabel('PC1')
    ax1.set_ylabel('PC2')
    ax1.set_title('PCA')

    # cumulative explained variance
    pca = PCA()
    pca.fit(X_scaled)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    ax2.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-')
    ax2.set_xlabel('Number of Principal Components')
    ax2.set_ylabel('Cumulative Explained Variance')
    percent_at_90 = ( np.argmax(cumulative_variance >= 0.9) + 1 ) / len(cumulative_variance)
    ax2.set_title(f'PCA Cumulative Explained Variance - Features at 90% variance: {percent_at_90:.2f}')
    ax2.grid(True)
    
    # t-SNE
    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    X_tsne = tsne.fit_transform(X_scaled)
    ax3.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y, cmap='coolwarm', alpha=0.5)
    ax3.set_title('t-SNE')

    '''
    If deep embeddings have lower skewness than fingerprints, they might distribute more evenly in feature space.
    If deep embeddings have higher positive skewness, they may collapse into tight clusters, potentially causing overfitting.

    High positive skewness → Many close points, some far → Possible overfitting / redundancy
    Low or negative skewness → More even spread → Potentially better representation
    If embeddings have too high or too low skewness, fine-tuning may help improve their representation quality.
    '''
    skewness = np.mean((spatial_distance - spatial_distance.mean()) ** 3) / (spatial_distance.std() ** 3)

    ax4.set_xlim(0, spatial_distance_max)
    sns.kdeplot(spatial_distance.flatten(), ax=ax4, fill=True, color='blue', alpha=0.5, )
    ax4.set_xlabel('Distance')
    ax4.set_ylabel('Density')
    ax4.set_title(f'Spatial distance - Mean: {spatial_distance.mean():.2f}, Std: {spatial_distance.std():.2f}, Skewness: {skewness:.2f}')

    # 


# Feature Variance Analysis
def analyze_variance(X):
    X_scaled = standardize(X)
    selector = VarianceThreshold()
    X_selected = selector.fit_transform(X_scaled)
    retained_features = X_selected.shape[1]
    return f'Low information fetures {X.shape[1] - retained_features}/{X.shape[1]}'

In [99]:
deep_learning_features = ['bartsmiles.csv', 'molformer.csv', 'megamolbart.csv', 'mol2vec.csv', 'grover.csv']

In [7]:
datasets_location = "/Users/orestisbastas/Developer/bio2d_public/data"
datasets = os.listdir(f"{datasets_location}/")

In [ ]:
count = 0
for dataset in sorted(datasets):
    if dataset.startswith("."):
        continue

    results = all_results[all_results['dataset'] == dataset]
    if results.empty:
        print(f"No results available for {dataset}")
        continue

    regression = False
    if results['task_type'].values[0] == 'regression':
        regression = True

    train_features = os.listdir(f'{datasets_location}/{dataset}/features/train/')

    results_with_means = {}

    fingerprints = {}
    deep_embeddings = {}
    spatial_distances = {}
    spatial_distance_max = 0
    y = None

    spatial_disances_location = f'/Users/orestisbastas/Developer/bio2d_public/results/spatial_distances/{dataset}.pkl'
    if os.path.isfile(spatial_disances_location):
        spatial_distances = pickle.load(open(spatial_disances_location, 'rb'))

    for train_feature in train_features:
        if 'rdkit_fp' in train_feature or 'autocorr_2d' in train_feature or 'mordred' in train_feature:
            continue
        feature_csv = pd.read_csv(f'{datasets_location}/{dataset}/features/train/{train_feature}')
        label = feature_csv['value']
        if y is None:
            y = label.values

        results_with_means[train_feature.split('.')[0]] = []

        # drop any column that doesn't have the train_feature prefix in the column name
        feature_csv = feature_csv.drop(columns=[col for col in feature_csv.columns if train_feature.split('.')[0] not in col])
        # get the features by combining every row as a numpy array
        features = feature_csv.values
    
        if spatial_distances.get(train_feature) is None:
            spatial_distance = pairwise_distances(features, metric='euclidean', n_jobs=-1)
            spatial_distance = spatial_distance.flatten()
        else:
            spatial_distance = spatial_distances[train_feature]
        if spatial_distance.max() > spatial_distance_max:
            spatial_distance_max = spatial_distance.max()
        spatial_distances[train_feature] = spatial_distance

        if train_feature in deep_learning_features:
            deep_embeddings[train_feature] = features
        else:
            fingerprints[train_feature] = features

    if not os.path.isfile(f'/Users/orestisbastas/Developer/bio2d_public/results/spatial_distances/{dataset}.pkl'):
        pickle.dump(spatial_distances, open(f'/Users/orestisbastas/Developer/bio2d_public/results/spatial_distances/{dataset}.pkl', 'wb'))

    all_features = {**fingerprints, **deep_embeddings}
    num_rows = len(all_features)
    fig = plt.figure(constrained_layout=True, figsize=(20, 5 * num_rows))
    fig.suptitle(f'PCA & t-SNE Analysis of Feature Representations for {dataset} dataset', fontsize=16, fontweight='bold')

    # Create subfigures
    subfigs = fig.subfigures(nrows=num_rows, ncols=1)
    for row, (name, X) in enumerate(all_features.items()):
        feature_name = name.split('.')[0]
        result_for_feature = {'regression': regression, 'feature_type': feature_name, 'rmse_val': None, 
                              'mae_val': None, 'nrmse_val': None, 'pearson_val': None, 'acc_val': None, 
                              'auprc_val': None, 'f1_val': None, 'spatial_distance_mean': None, 
                              'spatial_distance_std': None, 'fingerprint': True}
        if name.split('.')[0] in results['feature_type'].unique():
            result = results[results['feature_type'] == name.split('.')[0]].iloc[0]
            result_for_feature['rmse_val'] = result['rmse_val']
            result_for_feature['mae_val'] = result['mae_val']
            result_for_feature['nrmse_val'] = result['nrmse_val']
            result_for_feature['pearson_val'] = result['pearson_val']
            result_for_feature['acc_val'] = result['acc_val']
            result_for_feature['auprc_val'] = result['auprc_val']
            result_for_feature['f1_val'] = result['f1_val']
            if regression:
                results_string = f'RMSE {result["rmse_val"]:.2f} - MAE {result["mae_val"]:.2f} - NRMSE {result["nrmse_val"]:.2f} - Pearson {result["pearson_val"]:.2f}'
            else:
                results_string = f'Acc {result["acc_val"]:.2f} - AUC {result["auprc_val"]:.2f} - F1 {result["f1_val"]:.2f}'
        else:
            results_string = "No results available"
            
        sd = spatial_distances[name]
        result_for_feature['spatial_distance_mean'] = sd.mean()
        result_for_feature['spatial_distance_std'] = sd.std()
        feature_type = "Fingerprint" if name in fingerprints else "Deep Embedding"
        result_for_feature['fingerprint'] = feature_type == "Fingerprint"
        variance_info = analyze_variance(X)
        row_title = f'{name.split(".csv")[0]} ({feature_type}) - {variance_info} - {results_string}'
        subfigs[row].suptitle(row_title, fontsize=14, fontweight='bold')
        
        axs = subfigs[row].subplots(nrows=1, ncols=4)
        plot_pca_tsne(X, axs[0], axs[1], axs[2], axs[3], sd, spatial_distance_max)

        results_with_means[feature_name] = result_for_feature

    plt.show()

    # plot from results_with_means, for this dataset, correlation between spatial disance mean and mae_val and color code based on 'fingerprint' column
    spatial_distance_means = []
    mae_vals = []
    fingerprint_bool = []
    fingerprint_label = []
    for train_feature, result_details in results_with_means.items():
        if regression:
            if result_details['mae_val'] is not None:
                spatial_distance_means.append(result_details['spatial_distance_mean'])
                mae_vals.append(result_details['mae_val'])
                fingerprint_bool.append(result_details['fingerprint'])
                fingerprint_label.append('Fingerprint' if result_details['fingerprint'] else 'Deep Embedding')
        else:
            if result_details['acc_val'] is not None:
                spatial_distance_means.append(result_details['spatial_distance_mean'])
                mae_vals.append(result_details['acc_val'])
                fingerprint_bool.append(result_details['fingerprint'])
                fingerprint_label.append('Fingerprint' if result_details['fingerprint'] else 'Deep Embedding')
    spatial_distance_means = np.array(spatial_distance_means)
    mae_vals = np.array(mae_vals)
    plt.figure(figsize=(10, 5))
    scatter = plt.scatter(spatial_distance_means, mae_vals, c=fingerprint_bool, cmap='coolwarm', alpha=0.7, label=fingerprint_label)
    plt.xlabel('Spatial Distance Mean')
    plt.ylabel(f'{"MAE" if regression else "ACC"} Value')
    plt.title(f'Spatial Distance Mean vs {"MAE" if regression else "ACC"} Value for {dataset} dataset')
    plt.legend(fingerprint_label)
    plt.grid()
    plt.show()

    # count += 1
    # if count >= 1:
    #     break

